# Create Submission File

## Imports and get data

In [6]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
import numpy as np
from HelpFunctions.calc_score import evaluate_horizon
from HelpFunctions.mix_models import mix_models
from HelpFunctions.date_and_time import most_recent_thursday, split_time

from DAX.HelpFunctions.get_dax_data import get_dax_data
from DAX.Models import baseline as dax_bl
from DAX.Models import garch11 as dax_garch

from Energy.HelpFunctions.get_energy_data import get_energy_data, prepare_data
from Energy.Models import baseline as en_bl
from Energy.Models import Model4_population as en_m4_pop
from datetime import datetime
import pytz

# import importlib
# importlib.reload(Energy.Models)

# import importlib
# importlib.reload(Energy.Models)

### DAX: Get and prepare Data
Use tuesday as last datapoint

In [7]:
# Insert Date of Wednesday (forecast-day) here. The current day has to be wednesday!!
start_date = datetime(2023, 12, 20, 8, 0, 0)
assert start_date.weekday() == 2

In [8]:
df_dax = get_dax_data()
# pretend_start_date = date(2023, 11, 22, 11, 00, 0, tzinfo=pytz.timezone('Europe/Berlin'))

df_dax = df_dax[df_dax.index.date <= start_date.date()]

In [9]:
df_dax.tail()

,Close,ret1,ret2,ret3,ret4,ret5
Date,,,,,,
2023-12-14 00:00:00+01:00,16752.230469,-0.082464,-0.235570,-0.251585,-0.041718,0.738384
2023-12-15 00:00:00+01:00,16751.439453,-0.004722,-0.087186,-0.240292,-0.256307,-0.046440
2023-12-18 00:00:00+01:00,16650.550781,-0.604090,-0.608812,-0.691276,-0.844382,-0.860397
2023-12-19 00:00:00+01:00,16744.410156,0.562118,-0.041971,-0.046693,-0.129157,-0.282263
2023-12-20 00:00:00+01:00,16733.050781,-0.067863,0.494256,-0.109834,-0.114556,-0.197020


### Energy: Get and prepare Data

In [10]:
from Energy.HelpFunctions.get_energy_data import fetch_energy_data
fetch_energy_data()

  0%|          | 0/263 [00:00<?, ?it/s]/Users/elias/Desktop/PTSFC/Energy/HelpFunctions/get_energy_data.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 263/263 [00:22<00:00, 11.88it/s]


In [11]:
df_en = get_energy_data(force_return=True)
df_en = prepare_data(df_en)

In [12]:
df_en = df_en.loc[df_en.index <= start_date]

In [13]:
df_en.tail()

,gesamt
date_time,
2023-12-20 04:00:00,44.92275
2023-12-20 05:00:00,48.51875
2023-12-20 06:00:00,54.82350
2023-12-20 07:00:00,61.03775
2023-12-20 08:00:00,64.59875


## Run models for the data

### DAX

In [14]:
from DAX.Models import arma_garch_11 as ag
from DAX.Models import baseline as bl

import importlib
importlib.reload(ag)
importlib.reload(bl)


# dax_bl.baseline(df_dax)
# forecast_dax = dax_garch.garch11(df_dax)
forecast_dax = mix_models([ag.garch11_norm_1y, bl.baseline_1y], [1,1], df_dax, 'DAX')
forecast_dax


1
1
1
1
1


,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-12-21,DAX,1 day,-1.533723,-0.405423,0.041765,0.523728,1.532850
1,2023-12-22,DAX,2 day,-2.101241,-0.630386,0.054355,0.784434,2.280994
2,2023-12-25,DAX,5 day,-2.671243,-0.756009,0.119449,1.018722,2.715961
3,2023-12-26,DAX,6 day,-3.035672,-0.862041,0.154987,1.201198,3.249899
4,2023-12-27,DAX,7 day,-3.256062,-0.903910,0.194874,1.321378,3.602143


### Energy  

In [15]:
from Energy.Models.baseline import baseline
from HelpFunctions.mix_models import mix_models_per_horizon
from Energy.Models.Model4_sunhours import model4_sunhours
from Energy.Models.Model4_holidays_2 import model4_holidays_2
from Energy.Models.Model4_population import model4_population
from Energy.Models.Model5 import model5

# en_bl.baseline(df_en, LAST_IDX=-1, offset_horizons=11)

weights_m5_bl_m4_x = [1,1,0,0,0]
functions_m5_bl_m4_x = [model5, baseline, model4_population, model4_holidays_2, model4_sunhours]




# forecast_en = en_m4_pop.model4_population(df_en, offset_horizons=15)
forecast_en = mix_models(functions_m5_bl_m4_x, weights_m5_bl_m4_x, df_en, target="energy", offset_horizons=15)
forecast_en

AttributeError: 'DataFrame' object has no attribute 'weekday'

### Infections

In [ ]:
forecast_infections = {
    'forecast_date': [start_date.strftime('%Y-%m-%d')] * 5,
    'target': ['infections'] * 5,
    'horizon': ['0 week', '1 week', '2 week', '3 week', '4 week'],
    # 'q0.025': ['NA'] * 5,
    # 'q0.25': ['NA'] * 5,
    # 'q0.5': ['NA'] * 5,
    # 'q0.75': ['NA'] * 5,
    # 'q0.975': ['NA'] * 5,
    'q0.025': [np.nan] * 5,
    'q0.25': [np.nan] * 5,
    'q0.5': [np.nan] * 5,
    'q0.75': [np.nan] * 5,
    'q0.975': [np.nan] * 5
}

forecast_infections = pd.DataFrame(forecast_infections)

In [ ]:
forecast_infections

## Merge results

Check results: Are they valid, forecast_date is right? The last entry might be cut off

In [ ]:
forecasts = pd.concat([forecast_dax,forecast_en, forecast_infections]).reset_index().drop(columns=['index'])
forecasts

If everything seems good: Override forecast_date

In [ ]:
forecasts['forecast_date'] = [start_date.strftime('%Y-%m-%d') for _ in range(len(forecasts))]

In [ ]:
forecasts

### Check df & Save csv-File

In [ ]:
from HelpFunctions.check_submission import check_submission_df

check_submission_df(forecasts)
forecasts.to_csv(f'./Abgaben/{start_date.strftime("%Y%m%d")}_JakePeralta.csv', na_rep='NA', index=False)